# Mapping Yelp Search Results - Part 2

- 04/28/22

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [3]:
## Load in csv.gz
df = pd.read_csv("Data/Burgers_Baltimore.csv.gz")
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"{'latitude': 39.2771835822848, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '1041 Marshall St', 'address2': '...",1.443454e+10,(443) 453-9698,2161.212353
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 39.32696, 'longitude': -76.63753}","['delivery', 'pickup']",$$,"{'address1': '3300 Clipper Mill Rd', 'address2...",NaN,NaN,4242.702639
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 39.27174, 'longitude': -76.60223}","['delivery', 'pickup']",$$,"{'address1': '823 E Fort Ave', 'address2': '',...",1.410637e+10,(410) 637-3699,2754.995467
3,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,183,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 39.2774761241692, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '1032 Riverside Ave', 'address2':...",1.443389e+10,(443) 388-9113,2083.657647
4,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"{'latitude': 39.30715667, 'longitude': -76.625...","['delivery', 'pickup']",NaN,"{'address1': '1501 Bolton St', 'address2': Non...",1.410226e+10,(410) 225-9964,1990.078956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.33086, 'longitude': -76.63161}","['delivery', 'pickup']",$$,"{'address1': '907 W 36th St', 'address2': None...",1.410889e+10,(410) 889-3663,4382.210159
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,"{'latitude': 39.2821578979492, 'longitude': -7...",['delivery'],$,"{'address1': '1720 Thames St', 'address2': '',...",1.410558e+10,(410) 558-0399,2079.005086
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 39.28699, 'longitude': -76.56748}","['delivery', 'pickup']",$,"{'address1': '418 S Conkling St', 'address2': ...",1.410327e+10,(410) 327-4550,3607.859236
527,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,"{'latitude': 39.322087392211, 'longitude': -76...",['delivery'],NaN,"{'address1': '3320 Belair Rd', 'address2': '',...",1.410564e+10,(410) 563-6222,4134.435594


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [4]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 39.2771835822848, 'longitude': -7..."
1,"{'latitude': 39.32696, 'longitude': -76.63753}"
2,"{'latitude': 39.27174, 'longitude': -76.60223}"
3,"{'latitude': 39.2774761241692, 'longitude': -7..."
4,"{'latitude': 39.30715667, 'longitude': -76.625..."
...,...
524,"{'latitude': 39.33086, 'longitude': -76.63161}"
525,"{'latitude': 39.2821578979492, 'longitude': -7..."
526,"{'latitude': 39.28699, 'longitude': -76.56748}"
527,"{'latitude': 39.322087392211, 'longitude': -76..."


- Why didn't that work???

In [5]:
## slice out a single test coordinate
test_coord = df.loc[0,'coordinates']
test_coord

"{'latitude': 39.2771835822848, 'longitude': -76.6129885794452}"

In [6]:
type(test_coord)

str

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [7]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [8]:
## replace single ' with " 
test_coord = test_coord.replace("'", '"')
test_coord


'{"latitude": 39.2771835822848, "longitude": -76.6129885794452}'

In [9]:
## Use json.loads on the test coordinate, again
type(json.loads(test_coord))

dict

### Now, how can we apply this same process to the entire column??

In [10]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"{'latitude': 39.2771835822848, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '1041 Marshall St', 'address2': '...",1.443454e+10,(443) 453-9698,2161.212353
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 39.32696, 'longitude': -76.63753}","['delivery', 'pickup']",$$,"{'address1': '3300 Clipper Mill Rd', 'address2...",NaN,NaN,4242.702639
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"{'latitude': 39.27174, 'longitude': -76.60223}","['delivery', 'pickup']",$$,"{'address1': '823 E Fort Ave', 'address2': '',...",1.410637e+10,(410) 637-3699,2754.995467
3,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,183,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 39.2774761241692, 'longitude': -7...","['delivery', 'pickup']",$$,"{'address1': '1032 Riverside Ave', 'address2':...",1.443389e+10,(443) 388-9113,2083.657647
4,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"{'latitude': 39.30715667, 'longitude': -76.625...","['delivery', 'pickup']",NaN,"{'address1': '1501 Bolton St', 'address2': Non...",1.410226e+10,(410) 225-9964,1990.078956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"{'latitude': 39.33086, 'longitude': -76.63161}","['delivery', 'pickup']",$$,"{'address1': '907 W 36th St', 'address2': None...",1.410889e+10,(410) 889-3663,4382.210159
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,"{'latitude': 39.2821578979492, 'longitude': -7...",['delivery'],$,"{'address1': '1720 Thames St', 'address2': '',...",1.410558e+10,(410) 558-0399,2079.005086
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 39.28699, 'longitude': -76.56748}","['delivery', 'pickup']",$,"{'address1': '418 S Conkling St', 'address2': ...",1.410327e+10,(410) 327-4550,3607.859236
527,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,"{'latitude': 39.322087392211, 'longitude': -76...",['delivery'],NaN,"{'address1': '3320 Belair Rd', 'address2': '',...",1.410564e+10,(410) 563-6222,4134.435594


In [11]:
## slice out a single test coordinate
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,39.277184,-76.612989
1,39.326960,-76.637530
2,39.271740,-76.602230
3,39.277476,-76.609180
4,39.307157,-76.625848
...,...,...
524,39.330860,-76.631610
525,39.282158,-76.591797
526,39.286990,-76.567480
527,39.322087,-76.573110


### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [12]:
## use .apply pd.Series to convert a dict to columns
lat_lon = df['coordinates'].apply(pd.Series)
lat_lon

,latitude,longitude
0,39.277184,-76.612989
1,39.326960,-76.637530
2,39.271740,-76.602230
3,39.277476,-76.609180
4,39.307157,-76.625848
...,...,...
524,39.330860,-76.631610
525,39.282158,-76.591797
526,39.286990,-76.567480
527,39.322087,-76.573110


In [13]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df.drop(columns=['coordinates']),
           lat_lon], axis=1)

## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [14]:
## Load in mapbox api credentials from .secret
with open('/Users/codingdojo/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [15]:
## set mapbox token
px.set_mapbox_access_token(login['api-key'])

In [18]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df, lat='latitude', lon="longitude", mapbox_style='open-street-map',
                 color='rating')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [19]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df, lat='latitude', lon="longitude", mapbox_style='open-street-map',
                  hover_name='name',color='rating')

In [20]:
## add hover_name (name) and hover_data for price,rating,location
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df, lat='latitude', lon="longitude", mapbox_style='open-street-map',
                  hover_name='name', hover_data=['price','location','rating'],
                  color='rating')

### Fixing the Location Column

In [21]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

"{'address1': '1041 Marshall St', 'address2': '', 'address3': '', 'city': 'Baltimore', 'zip_code': '21230', 'country': 'US', 'state': 'MD', 'display_address': ['1041 Marshall St', 'Baltimore, MD 21230']}"

> Also a string-dictionary...

In [22]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230
3,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,183,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""1032 Riverside Ave"", ""address2"":...",1.443389e+10,(443) 388-9113,2083.657647,39.277476,-76.609180
4,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"['delivery', 'pickup']",NaN,"{""address1"": ""1501 Bolton St"", ""address2"": Non...",1.410226e+10,(410) 225-9964,1990.078956,39.307157,-76.625848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{""address1"": ""907 W 36th St"", ""address2"": None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", ""address2"": """",...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"['delivery', 'pickup']",$,"{""address1"": ""418 S Conkling St"", ""address2"": ...",1.410327e+10,(410) 327-4550,3607.859236,39.286990,-76.567480
527,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,['delivery'],NaN,"{""address1"": ""3320 Belair Rd"", ""address2"": """",...",1.410564e+10,(410) 563-6222,4134.435594,39.322087,-76.573110


In [23]:
## apply json.loads
df['location'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 66 (char 65)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [25]:
## slice out test address 
test_addr = df.loc[4, 'location']
test_addr

'{"address1": "1501 Bolton St", "address2": None, "address3": "", "city": "Baltimore", "zip_code": "21217", "country": "US", "state": "MD", "display_address": ["1501 Bolton St", "Baltimore, MD 21217"]}'

In [26]:
## write a function to just run json.loads on the address
def fix_address(addr):
    try:
        return json.loads(addr)
    except:
        return "ERROR"

In [27]:
## test applying our function
fix_address(test_addr)

'ERROR'

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [29]:
### save a new display_location column using our functiond
df['display_location'] = df['location'].apply(fix_address)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '..."
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530,ERROR
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',..."
3,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,183,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""1032 Riverside Ave"", ""address2"":...",1.443389e+10,(443) 388-9113,2083.657647,39.277476,-76.609180,ERROR
4,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"['delivery', 'pickup']",NaN,"{""address1"": ""1501 Bolton St"", ""address2"": Non...",1.410226e+10,(410) 225-9964,1990.078956,39.307157,-76.625848,ERROR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{""address1"": ""907 W 36th St"", ""address2"": None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610,ERROR
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", ""address2"": """",...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797,"{'address1': '1720 Thames St', 'address2': '',..."
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"['delivery', 'pickup']",$,"{""address1"": ""418 S Conkling St"", ""address2"": ...",1.410327e+10,(410) 327-4550,3607.859236,39.286990,-76.567480,"{'address1': '418 S Conkling St', 'address2': ..."
527,x6u_Xd5Zr6skcQsIUiUOng,jazzys-baltimore,Jazzy's,https://s3-media1.fl.yelpcdn.com/bphoto/NcDv9-...,False,https://www.yelp.com/biz/jazzys-baltimore?adju...,1,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",2.0,['delivery'],NaN,"{""address1"": ""3320 Belair Rd"", ""address2"": """",...",1.410564e+10,(410) 563-6222,4134.435594,39.322087,-76.573110,"{'address1': '3320 Belair Rd', 'address2': '',..."


In [30]:
## filter for businesses with display_location == "ERROR"
df.loc[ df['display_location']=='ERROR']

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
1,3j6GTI0V2Jshw8e1c3uFoA,the-urban-burger-bar-baltimore,The Urban Burger Bar,https://s3-media1.fl.yelpcdn.com/bphoto/mmtgGp...,False,https://www.yelp.com/biz/the-urban-burger-bar-...,32,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""3300 Clipper Mill Rd"", ""address2...",NaN,NaN,4242.702639,39.326960,-76.637530,ERROR
3,jvKhto6__tCGfs2N27oWdg,the-outpost-american-tavern-baltimore-2,The Outpost American Tavern,https://s3-media2.fl.yelpcdn.com/bphoto/4bN4vA...,False,https://www.yelp.com/biz/the-outpost-american-...,183,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""1032 Riverside Ave"", ""address2"":...",1.443389e+10,(443) 388-9113,2083.657647,39.277476,-76.609180,ERROR
4,5CM8nuqd0od68GqQ-UGicQ,cookhouse-baltimore-2,CookHouse,https://s3-media2.fl.yelpcdn.com/bphoto/N-2JWj...,False,https://www.yelp.com/biz/cookhouse-baltimore-2...,20,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"['delivery', 'pickup']",NaN,"{""address1"": ""1501 Bolton St"", ""address2"": Non...",1.410226e+10,(410) 225-9964,1990.078956,39.307157,-76.625848,ERROR
6,lx6ZQjgZDvL0Rz9LmXtTKg,charmed-baltimore,Charmed.,https://s3-media2.fl.yelpcdn.com/bphoto/e9AV7X...,False,https://www.yelp.com/biz/charmed-baltimore?adj...,72,"[{'alias': 'newamerican', 'title': 'American (...",5.0,"['delivery', 'pickup']",NaN,"{""address1"": ""824 N Calvert St"", ""address2"": N...",1.443835e+10,(443) 835-2803,609.329117,39.299510,-76.613340,ERROR
10,LkCSwCV-WX2eCnv5RUM3Kw,311-west-madison-ave-craft-beer-and-wine-resta...,311 West Madison Ave Craft Beer & Wine Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/g7er6A...,False,https://www.yelp.com/biz/311-west-madison-ave-...,6,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['delivery', 'pickup']",NaN,"{""address1"": ""311 W Madison St"", ""address2"": N...",1.443939e+10,(443) 938-9109,1141.160510,39.298216,-76.620656,ERROR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,p8_yO34vsydnG-AgoIyIMA,canton-pizza-and-deli-baltimore,Canton Pizza & Deli,https://s3-media1.fl.yelpcdn.com/bphoto/WO2KWf...,False,https://www.yelp.com/biz/canton-pizza-and-deli...,83,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['pickup', 'delivery']",$,"{""address1"": ""3300 O""Donnell St"", ""address2"": ...",1.410564e+10,(410) 563-7777,3716.916969,39.280613,-76.569435,ERROR
518,4QrEtrKk8A2o1CclzKcKqg,ny-chicken-and-grill-baltimore-2,NY Chicken & Grill,NaN,False,https://www.yelp.com/biz/ny-chicken-and-grill-...,2,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.0,"['pickup', 'delivery']",NaN,"{""address1"": ""2101 W Pratt St"", ""address2"": """"...",1.443709e+10,(443) 708-5338,3860.731194,39.284810,-76.650040,ERROR
521,gXQL3Zmlv0aXzjhYPRt5tg,mcdonalds-baltimore-57,McDonald's,https://s3-media2.fl.yelpcdn.com/bphoto/VAqcvo...,False,https://www.yelp.com/biz/mcdonalds-baltimore-5...,6,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",1.5,['delivery'],$,"{""address1"": ""4526 Erdman Ave"", ""address2"": No...",1.410484e+10,(410) 483-9723,4193.333190,39.309368,-76.562009,ERROR
524,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.5,"['delivery', 'pickup']",$$,"{""address1"": ""907 W 36th St"", ""address2"": None...",1.410889e+10,(410) 889-3663,4382.210159,39.330860,-76.631610,ERROR


In [31]:
## slice out a new test address and inspect
test_addr = df.loc[437, 'location']
test_addr

'{"address1": "2839 O"Donnell St", "address2": "", "address3": "", "city": "Baltimore", "zip_code": "21224", "country": "US", "state": "MD", "display_address": ["2839 O"Donnell St", "Baltimore, MD 21224"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [34]:
## remove any rows where display_location == 'ERROR'
df = df.loc[ ~(df['display_location']=='ERROR')]
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '..."
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',..."
5,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,266,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749,"{'address1': '811 S Broadway', 'address2': '',..."
7,WRz58h4VB9nadcyar4LGvQ,wet-city-baltimore,Wet City,https://s3-media4.fl.yelpcdn.com/bphoto/8L30rM...,False,https://www.yelp.com/biz/wet-city-baltimore?ad...,159,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.0,['delivery'],$$,"{""address1"": ""223 W Chase St"", ""address2"": """",...",1.443874e+10,(443) 873-6699,1143.873380,39.301609,-76.618950,"{'address1': '223 W Chase St', 'address2': '',..."
8,6PLtSs9Bi4QDSNBM_u4fNg,the-brewers-art-baltimore,The Brewer's Art,https://s3-media1.fl.yelpcdn.com/bphoto/2pkJFg...,False,https://www.yelp.com/biz/the-brewers-art-balti...,1039,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1106 N Charles St"", ""address2"": ...",1.410548e+10,(410) 547-6925,1040.671677,39.302825,-76.616180,"{'address1': '1106 N Charles St', 'address2': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,WVQqkLAKowuIe51IVNXCZA,outback-steakhouse-baltimore,Outback Steakhouse,https://s3-media2.fl.yelpcdn.com/bphoto/NzwBx9...,False,https://www.yelp.com/biz/outback-steakhouse-ba...,135,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3.0,"['delivery', 'pickup']",$$,"{""address1"": ""2400 Boston St"", ""address2"": """",...",1.410523e+10,(410) 522-7757,2755.986468,39.281747,-76.581631,"{'address1': '2400 Boston St', 'address2': '',..."
523,L1915YsFXYdkuXMwNbBwog,harris-teeter-baltimore-9,Harris Teeter,https://s3-media2.fl.yelpcdn.com/bphoto/ILRLQX...,False,https://www.yelp.com/biz/harris-teeter-baltimo...,100,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.0,[],$$,"{""address1"": ""3779 Boston St"", ""address2"": """",...",1.410522e+10,(410) 522-0270,4366.139708,39.275148,-76.564825,"{'address1': '3779 Boston St', 'address2': '',..."
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", ""address2"": """",...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797,"{'address1': '1720 Thames St', 'address2': '',..."
526,vnOzM6USxuhmuXV_uqg0JA,filippos-baltimore,Filippo's,https://s3-media2.fl.yelpcdn.com/bphoto/NiWpYq...,False,https://www.yelp.com/biz/filippos-baltimore?ad...,51,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"['delivery', 'pickup']",$,"{""address1"": ""418 S Conkling St"", ""address2"": ...",1.410327e+10,(410) 327-4550,3607.8592

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [36]:
## use apply and lambda to slice correct key
test_loc = df.loc[0,"display_location"]
test_loc

{'address1': '1041 Marshall St',
 'address2': '',
 'address3': '',
 'city': 'Baltimore',
 'zip_code': '21230',
 'country': 'US',
 'state': 'MD',
 'display_address': ['1041 Marshall St', 'Baltimore, MD 21230']}

In [42]:
print("\n".join(test_loc['display_address']))

1041 Marshall St
Baltimore, MD 21230


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [43]:
df['display_address'] = df['display_location'].apply(lambda x: '\n'.join(x['display_address']))
df

/var/folders/rf/vw4r41jd7vd95x1w0dth7v9h0000gp/T/ipykernel_36934/521540563.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude,display_location,display_address
0,AQ56plNP56TIk3JswtoiMA,abbey-burger-bistro-baltimore-4,Abbey Burger Bistro,https://s3-media4.fl.yelpcdn.com/bphoto/_ZZ4nI...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,914,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1041 Marshall St"", ""address2"": ""...",1.443454e+10,(443) 453-9698,2161.212353,39.277184,-76.612989,"{'address1': '1041 Marshall St', 'address2': '...","1041 Marshall St\nBaltimore, MD 21230"
2,QHVUhI8JBAcqGj1JufDcHw,wiley-gunters-baltimore,Wiley Gunters,https://s3-media3.fl.yelpcdn.com/bphoto/Lc-4k9...,False,https://www.yelp.com/biz/wiley-gunters-baltimo...,135,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.5,"['delivery', 'pickup']",$$,"{""address1"": ""823 E Fort Ave"", ""address2"": """",...",1.410637e+10,(410) 637-3699,2754.995467,39.271740,-76.602230,"{'address1': '823 E Fort Ave', 'address2': '',...","823 E Fort Ave\nBaltimore, MD 21230"
5,DnhKT0A9OIgFdIhzNryJBw,abbey-burger-bistro-baltimore-6,Abbey Burger Bistro,https://s3-media3.fl.yelpcdn.com/bphoto/oq4-Iy...,False,https://www.yelp.com/biz/abbey-burger-bistro-b...,266,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""811 S Broadway"", ""address2"": """",...",1.410522e+10,(410) 522-1428,2009.275703,39.282278,-76.592749,"{'address1': '811 S Broadway', 'address2': '',...","811 S Broadway\nBaltimore, MD 21231"
7,WRz58h4VB9nadcyar4LGvQ,wet-city-baltimore,Wet City,https://s3-media4.fl.yelpcdn.com/bphoto/8L30rM...,False,https://www.yelp.com/biz/wet-city-baltimore?ad...,159,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.0,['delivery'],$$,"{""address1"": ""223 W Chase St"", ""address2"": """",...",1.443874e+10,(443) 873-6699,1143.873380,39.301609,-76.618950,"{'address1': '223 W Chase St', 'address2': '',...","223 W Chase St\nBaltimore, MD 21201"
8,6PLtSs9Bi4QDSNBM_u4fNg,the-brewers-art-baltimore,The Brewer's Art,https://s3-media1.fl.yelpcdn.com/bphoto/2pkJFg...,False,https://www.yelp.com/biz/the-brewers-art-balti...,1039,"[{'alias': 'breweries', 'title': 'Breweries'},...",4.0,"['delivery', 'pickup']",$$,"{""address1"": ""1106 N Charles St"", ""address2"": ...",1.410548e+10,(410) 547-6925,1040.671677,39.302825,-76.616180,"{'address1': '1106 N Charles St', 'address2': ...","1106 N Charles St\nBaltimore, MD 21201"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,WVQqkLAKowuIe51IVNXCZA,outback-steakhouse-baltimore,Outback Steakhouse,https://s3-media2.fl.yelpcdn.com/bphoto/NzwBx9...,False,https://www.yelp.com/biz/outback-steakhouse-ba...,135,"[{'alias': 'steak', 'title': 'Steakhouses'}]",3.0,"['delivery', 'pickup']",$$,"{""address1"": ""2400 Boston St"", ""address2"": """",...",1.410523e+10,(410) 522-7757,2755.986468,39.281747,-76.581631,"{'address1': '2400 Boston St', 'address2': '',...","2400 Boston St\nBaltimore, MD 21224"
523,L1915YsFXYdkuXMwNbBwog,harris-teeter-baltimore-9,Harris Teeter,https://s3-media2.fl.yelpcdn.com/bphoto/ILRLQX...,False,https://www.yelp.com/biz/harris-teeter-baltimo...,100,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.0,[],$$,"{""address1"": ""3779 Boston St"", ""address2"": """",...",1.410522e+10,(410) 522-0270,4366.139708,39.275148,-76.564825,"{'address1': '3779 Boston St', 'address2': '',...","3779 Boston St\nBaltimore, MD 21224"
525,cA-DS-nJlfl7IYre-rjH4Q,daily-grind-baltimore-8,Daily Grind,https://s3-media1.fl.yelpcdn.com/bphoto/jrsOvc...,False,https://www.yelp.com/biz/daily-grind-baltimore...,231,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],$,"{""address1"": ""1720 Thames St"", ""address2"": """",...",1.410558e+10,(410) 558-0399,2079.005086,39.282158,-76.591797,"{'address1': '1720 Thames St', 'address2': '',...","1720 Thames St\nBaltimore, MD 21231"
526,vnOzM6USxu

### Final Map

In [44]:
## make ourn final map and save as varaible
px.scatter_mapbox(df, lat='latitude', lon="longitude", mapbox_style='open-street-map',
                  hover_name='name', hover_data=['price','display_address','rating'],
                  color='rating')

#### HTML Uses `<br>` instead of `\n`

In [48]:
## remake the final address column with <br> instead 
df['display_address'] = df['display_location'].apply(lambda x: '<br>'.join(x['display_address']))
## plot the final map
## make ourn final map and save as varaible
fig= px.scatter_mapbox(df, lat='latitude', lon="longitude", mapbox_style='open-street-map',
                  hover_name='name', hover_data=['price','display_address','rating'],
                  color='rating')
fig.show()

/var/folders/rf/vw4r41jd7vd95x1w0dth7v9h0000gp/T/ipykernel_36934/1645063056.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [50]:
## use fig.write_html to save map
fig.write_html('final_map_from_class.html')